In [1]:
import numpy as np
import pandas as pd

import matplotlib as plt

# <1> Permutation entropy
- 순열 엔트로피(Permutation entropy)는 어떤 순열의 변화의 정도를 측정하는 척도이며, 순열 엔트로피는 어떤 시계열 자료의 오르고 내리는 변동이 얼마나 무작위적인지를 측정하여 분석하는 데에 사용된다.
- 순열 엔트로피는 수열, 즉 시계열 데이터가 있을 때 연속된 k개의 데이터의 순위 변화를 나타내는 tuple의 엔트로피로 결정된다.
<br><br>
- 예를 들어 $x = (4,7,9,10,6,11,3)$이고 $k=3$이라면 가장 앞에 있는 세 개의 원소 $(4,7,9)$의 순위(rank)는 $(1,2,3)$이다. 그 다음 3원소 $(7,9,10)$의 순위는 $(1,2,3)$이다.
- 이에 따라 5개의 rank tuple을 구한다면 $(1,2,3), (1,2,3), (2,3,1), (2,1,3), (2,3,1)$이 된다. 
- 이 tuple을 기준으로 entropy를 구한다면 $(1,2,3)$과 $(2,3,1)$은 각각 두 번, $(2,1,3)$은 한 번 발생하므로 각 tuple의 확률은 $2/5, 2/5, 1/5$이며, k=3인 수열 x의 permutation entropy는 다음과 같이 계산된다.
$$PE(x,3)=-(2/5)log(2/5)-(2/5)log(2/5)-1/5log(1/5)$$


## 1. 코드 구현
- 순열 엔트로피의 계산 단계를 코드로 구현한다면, 아래와 같이 세 단계로 나뉜다.

### 1.1 Chunk the data into sub-windows of length D starting every tau.

x = [4, 7, 9, 10, 6, 11, 3]는 D = 3일 때 아래와 같이 나눠진다.
(D는 앞서 말한 k에 대응된다.)

In [2]:
x=[4, 7, 9, 10, 6, 11, 3]

def chunk_data(x, D):
    chunked_x = []
    for i in range(0,len(x)-D+1):
        chunked_x.append(x[i:i+D])
    return chunked_x

print(chunk_data(x, 3))


[[4, 7, 9], [7, 9, 10], [9, 10, 6], [10, 6, 11], [6, 11, 3]]


### 1.2 Replace each D-window by the permutation, that captures the ordinal ranking of the data
- D 사이즈의 chunk로 나뉜 각각의 튜플을 tuple 내의 rank로 표현한다

In [3]:
chunked_x = chunk_data(x,3)

permutations = np.argsort(np.argsort(chunked_x))
permutations

array([[0, 1, 2],
       [0, 1, 2],
       [1, 2, 0],
       [1, 0, 2],
       [1, 2, 0]])

### 1.3 Count the frequencies of every permutation and return their entropy (we use log_e and not log_2).
- 각 튜플의 발생 횟수를 세고, 각 횟수에 따른 발생 확률을 구한다.
- 해당 확률을 기반으로 entropy를 계산한다

In [4]:
# Count the number of occurences
tuples, counts = np.unique(permutations, axis=0, return_counts=True)
tuples, counts

(array([[0, 1, 2],
        [1, 0, 2],
        [1, 2, 0]]),
 array([2, 1, 2]))

In [5]:
# turn them into frequencies
probs = counts / len(permutations)
probs

array([0.4, 0.2, 0.4])

In [6]:
# Calculate entropy
pe = -np.sum(probs * np.log(probs))
pe

1.0549201679861442

## 2. TSFRESH 구현 코드

- TSFRESH 라이브러리 내 구현 코드는 아래와 같다. (https://tsfresh.readthedocs.io/en/latest/_modules/tsfresh/feature_extraction/feature_calculators.html#permutation_entropy) 
- 추가적으로 tau라는 변수가 고려된다. tau는 step을 의미한다.

In [7]:
def permutation_entropy(x, tau, dimension):
    """
    Calculate the permutation entropy.

    Three steps are needed for this:

    1. chunk the data into sub-windows of length D starting every tau.
       Following the example from the reference, a vector

        x = [4, 7, 9, 10, 6, 11, 3]

       with D = 3 and tau = 1 is turned into

           [[ 4,  7,  9],
            [ 7,  9, 10],
            [ 9, 10,  6],
            [10,  6, 11],
            [ 6, 11,  3]]

    2. replace each D-window by the permutation, that
       captures the ordinal ranking of the data.
       That gives

           [[0, 1, 2],
            [0, 1, 2],
            [1, 2, 0],
            [1, 0, 2],
            [1, 2, 0]]

    3. Now we just need to count the frequencies of every permutation
       and return their entropy (we use log_e and not log_2).

    Ref: https://www.aptech.com/blog/permutation-entropy/
         Bandt, Christoph and Bernd Pompe.
         “Permutation entropy: a natural complexity measure for time series.”
         Physical review letters 88 17 (2002): 174102 .
    """

    X = _into_subchunks(x, dimension, tau)
    if len(X) == 0:
        return np.nan
    # Now that is clearly black, magic, but see here:
    # https://stackoverflow.com/questions/54459554/numpy-find-index-in-sorted-array-in-an-efficient-way
    permutations = np.argsort(np.argsort(X))
    # Count the number of occurences
    _, counts = np.unique(permutations, axis=0, return_counts=True)
    # turn them into frequencies
    probs = counts / len(permutations)
    # and return their entropy
    return -np.sum(probs * np.log(probs))


def _into_subchunks(x, subchunk_length, every_n=1):
    """
    Split the time series x into subwindows of length "subchunk_length", starting every "every_n".

    For example, the input data if [0, 1, 2, 3, 4, 5, 6] will be turned into a matrix

        0  2  4
        1  3  5
        2  4  6

    with the settings subchunk_length = 3 and every_n = 2
    """
    len_x = len(x)

    assert subchunk_length > 1
    assert every_n > 0

    # how often can we shift a window of size subchunk_length over the input?
    num_shifts = (len_x - subchunk_length) // every_n + 1
    shift_starts = every_n * np.arange(num_shifts)
    indices = np.arange(subchunk_length)

    indexer = np.expand_dims(indices, axis=0) + np.expand_dims(shift_starts, axis=1)
    return np.asarray(x)[indexer]

동일한 순열에 대한 entropy 값을 구해보면 아래와 같이 동일하게 나오는 것을 확인할 수 있다

In [8]:
permutation_entropy([4, 7, 9, 10, 6, 11, 3], 1, 3)

1.0549201679861442

## 3. 실제 데이터 적용

In [9]:
series = pd.read_csv('./data/daily-min-temperatures.csv', header=0, index_col=0)
print(series.head())
print(series.shape)

            Temp
Date            
1981-01-01  20.7
1981-01-02  17.9
1981-01-03  18.8
1981-01-04  14.6
1981-01-05  15.8
(3650, 1)


Calculate the permutation entropy for temperature with the window size of 3 and the shift of 1


In [10]:
temperatures = series['Temp'].values
permutation_entropy(temperatures, 1, 3)

1.7578877034206861

다른 window size에 따른 permutation entropy 값을 확인해보자

In [11]:
windows = [3, 5, 10, 50, 100, 3650]
for w in windows:
    print(f"window size: {w} | entropy: {permutation_entropy(temperatures, 1, w):.4f}")

window size: 3 | entropy: 1.7579
window size: 5 | entropy: 4.5964
window size: 10 | entropy: 8.1939
window size: 50 | entropy: 8.1890
window size: 100 | entropy: 8.1750
window size: 3650 | entropy: -0.0000


k가 증가함에 따라서 entropy가 변화하며, window size가 total length와 동일할 때 entropy는 0이다.<br>
entropy가 가장 높은 경우는 나타날 수 있는 모든 순위의 변화,즉 k!개의 모든 순위가 나타날 때임을 알 수 있다.<br>
permutation entropy는 주가 변동 분석 등 시계열 분석에 있어서 어떤 데이터의 오르내리는 변동의 무작위성을 측정할 수 있는 도구이다.

Reference
- https://www.aptech.com/blog/permutation-entropy/